In [1]:
!pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.4/484.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 49.9 MB/s eta 0:

In [2]:
!pip install botorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.8/567.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.8/732.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 kB 10.6 MB/s eta 0:00:00


In [3]:
import sklearn
import scipy
from pycaret.regression import *
import botorch
import torch
import gpytorch
import pandas as pd
import numpy as np

# Linear Regression com normalização

In [4]:
df=pd.DataFrame([["AISI_1020",0.5,99999,99999,5,9999999,213,53,0.0055,0.34,0.81,2.70E-03,206000,0.28],
              ["AISI_1045",0.5,99999,99999,5,9999999,553,600,0.0134,0.234,1.000,2.70E-03,210000,0.269],
              ["AISI_4340",0.5,99999,99999,5,9999999,792,510,0.014,1.030,0.26,2.70E-03,209000,0.28],
              ["AISI_4340_52_HRC",0.5,99999,99999,5,9999999,1638,1034,0.0058,0.46,1.003,2.70E-03,222000,0.29],
              ["STEEL_304",0.5,99999,99999,5,9999999,310,1000,0.07,0.65,1.000,2.70E-03,209000,0.28],
              ["STEEL_52100",0.5,99999,99999,5,9999999,774.78,134,0.0018,0.37,3.171,2.70E-03,222000,0.29]],
              columns=["material","depth","vc","rake_angle","clearance_angle","expected_cut_force",
                        "A","B","C","n","m","densidade","young_modulus","poisson_coef"])
df

,material,depth,vc,rake_angle,clearance_angle,expected_cut_force,A,B,C,n,m,densidade,young_modulus,poisson_coef
0,AISI_1020,0.5,99999,99999,5,9999999,213.00,53,0.0055,0.340,0.810,0.0027,206000,0.280
1,AISI_1045,0.5,99999,99999,5,9999999,553.00,600,0.0134,0.234,1.000,0.0027,210000,0.269
2,AISI_4340,0.5,99999,99999,5,9999999,792.00,510,0.0140,1.030,0.260,0.0027,209000,0.280
3,AISI_4340_52_HRC,0.5,99999,99999,5,9999999,1638.00,1034,0.0058,0.460,1.003,0.0027,222000,0.290
4,STEEL_304,0.5,99999,99999,5,9999999,310.00,1000,0.0700,0.650,1.000,0.0027,209000,0.280
5,STEEL_52100,0.5,99999,99999,5,9999999,774.78,134,0.0018,0.370,3.171,0.0027,222000,0.290


In [5]:
def build_input(controlable_input, fixed_input,construct_dict):
    final_input_df=pd.DataFrame()
    for input in controlable_input:
        for counter,parameter in enumerate(construct_dict.keys()):
            fixed_input[parameter]=input[counter]
        final_input_df=pd.concat([final_input_df,fixed_input])
    return final_input_df

In [6]:
construct_dict={"rake_angle":{"sequence_len":4,"max_bound":10,"lower_bound":-10,"type":"int"},
                "vc":{"sequence_len":10,"max_bound":500,"lower_bound":30,"type":"int"},
                #"depth":{"sequence_len":4,"max_bound":2,"lower_bound":0.1,"type":"float"}
                }

In [7]:
degree_to_radian=lambda x: x*np.pi/180
radians_to_degree=lambda x: x*180/np.pi

# Minimização da força de corte

In [8]:
lasso_v0=load_model("models/lasso_v0")
lgbm_v0=load_model("models/lgbm_v0")
lgbm_v1=load_model("models/lgbm_v1")

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


In [9]:
def objective_func(x,iloc_=0,model="lasso"):
    #x -> [[rake_angle,vc]]
    x=[x]
    inputs=build_input(x,df.iloc[[iloc_]],construct_dict)
    if model=="lasso":
      model_art=lasso_v0
    else:
      model_art=lgbm_v1
    cut_force=(predict_model(model_art,inputs,verbose=False).drop_duplicates()["prediction_label"].values)
    return cut_force[0]

### Força bruta

In [10]:
import itertools

# initialize lists
list_1 = [str(x) for x in np.linspace(-15,15,10)]
list_2 = [str(x) for x in np.linspace(2500,6500,10)]

unique_combinations = []

permut = itertools.product(list_1,list_2)
for comb in permut:
    unique_combinations.append(comb)

flat_list = unique_combinations

In [11]:
len(flat_list)

100

In [12]:
from tqdm import tqdm
dict_={"rake_angle":[],"vc":[],"fcut_lasso":[],"fcut_lgbm":[],"material":[]}
count=0
for i in tqdm(flat_list):
    count+=1
    for j in range(df.shape[0]):
        dict_["rake_angle"].append(i[0])
        dict_["vc"].append(i[1])
        dict_["fcut_lasso"].append(objective_func(i,j,"lasso"))
        dict_["fcut_lgbm"].append(objective_func(i,j,"lgbm"))
        dict_["material"].append(df.iloc[j]["material"])
    #if count>5:
      #break
results=pd.DataFrame.from_dict(dict_)
results.head()

100%|██████████| 100/100 [06:11<00:00,  3.71s/it]


,rake_angle,vc,fcut_lasso,fcut_lgbm,material
0,-15.0,2500.0,1422.019841,1040.572327,AISI_1020
1,-15.0,2500.0,5816.711489,5791.133299,AISI_1045
2,-15.0,2500.0,6792.855149,6265.977035,AISI_4340
3,-15.0,2500.0,22655.168641,6272.179365,AISI_4340_52_HRC
4,-15.0,2500.0,2657.087847,2135.276159,STEEL_304


In [13]:
results.sort_values("fcut_lasso").drop_duplicates("material")

,rake_angle,vc,fcut_lasso,fcut_lgbm,material
354,1.6666666666666679,6500.0,543.806075,902.413834,AISI_1020
358,1.6666666666666679,6500.0,1739.561695,2075.995596,STEEL_304
419,5.0,6500.0,2434.555144,2836.419103,STEEL_52100
415,5.0,6500.0,4646.259750,4495.768292,AISI_1045
416,5.0,6500.0,5634.816978,6148.112747,AISI_4340
537,11.666666666666668,6500.0,20729.504855,5846.490428,AISI_4340_52_HRC


In [14]:
results.sort_values("fcut_lgbm").drop_duplicates("material")

,rake_angle,vc,fcut_lasso,fcut_lgbm,material
318,1.6666666666666679,3833.3333333333335,824.452877,891.098374,AISI_1020
142,-8.333333333333332,3833.3333333333335,2209.573384,1968.813921,STEEL_304
329,1.6666666666666679,4277.777777777777,2686.772805,2734.099076,STEEL_52100
385,5.0,4277.777777777777,4890.191110,4495.768292,AISI_1045
363,5.0,2500.0,21203.589554,5707.738931,AISI_4340_52_HRC
548,15.0,2944.4444444444443,6216.350140,5747.069291,AISI_4340


In [15]:
import plotly.express as px

results["vc_m/min"]=results["vc"].astype(float)*60/1000
results["rake_angle"]=results["rake_angle"].astype(float)
results["fcut_lasso"]=results["fcut_lasso"].astype(float)
results["fcut_lgbm"]=results["fcut_lgbm"].astype(float)
for material in results["material"].unique():
  for model_ in ["lasso","lgbm"]:
    fig = px.scatter(results[results["material"]==material],
                    x="rake_angle",
                    y=f"fcut_{model_}",
                    color="vc_m/min",
                    title=f"Força de corte prevista por velocidade de corte e ângulo de saída para o {material} com {model_}",
                    )

    fig.update_layout(
                    xaxis_title="Ângulo de saída [°]",
                    yaxis_title="Força de corte [N]",
                    legend_title="Velocidade de corte [m/min]",
      )
    fig.show()

## Método de minimização

In [41]:
from scipy.optimize import minimize

x_iteracoes={"model":[],"material":[],"x":[]}
iteracoes_=[]
def save_iteractions(x):
  global iteracoes_
  iteracoes_.append(x)

bounds=[(-15,15),(3000,5000)]

results_powell=pd.DataFrame()
count=0
for model_ in ["lgbm","lasso"]:
  for idx_mat,material in enumerate(df["material"].unique()):
    iteracoes_=[]
    count+=1
    output=minimize(objective_func,
                    args=(idx_mat,model_),
                    x0=[0,3500],
                    method="Powell",
                    bounds=bounds,
                    callback=save_iteractions)
    x_iteracoes["x"].append(iteracoes_)
    x_iteracoes["model"].append(model_)
    x_iteracoes["material"].append(material)
    results_powell=pd.concat([
        results_powell,
        pd.DataFrame(output.items()).set_index(0).T.assign(material=df["material"].unique()[idx_mat],
                                                           model=model_)
        ])

results["vc"]=(results["vc"].astype(float)*60/1000)

In [44]:
x_iteracoes=pd.DataFrame.from_dict(x_iteracoes)

In [36]:
results_powell

,fun,direc,nit,nfev,status,success,message,x,material,model
1,956.912258,"[[1.0, 0.0], [0.0, 1.0]]",1,57,0,True,Optimization terminated successfully.,"[-6.308369924161681, 3763.932063670938]",AISI_1020,lgbm
1,4207.842781,"[[0.0, 1.0], [0.2264288063448883, 93.762053769...",2,130,0,True,Optimization terminated successfully.,"[2.708658287922292, 4527.8680086679415]",AISI_1045,lgbm
1,5847.703923,"[[1.0, 0.0], [0.813273430987238, 375.649959989...",3,161,0,True,Optimization terminated successfully.,"[1.375661608804293, 4143.348309797111]",AISI_4340,lgbm
1,5874.275986,"[[0.0, 1.0], [-1.6984525267336017, -714.859236...",3,187,0,True,Optimization terminated successfully.,"[0.9457816721851592, 4527.823877355981]",AISI_4340_52_HRC,lgbm
1,1947.86663,"[[1.0, 0.0], [0.0, 1.0]]",2,125,0,True,Optimization terminated successfully.,"[-3.7551906973150313, 3763.9320675856834]",STEEL_304,lgbm
1,2793.023499,"[[1.0, 0.0], [0.0, 1.0]]",1,61,0,True,Optimization terminated successfully.,"[10.623114247710914, 4527.864123305692]",STEEL_52100,lgbm
1,716.999017,"[[1.0, 0.0], [-3.143740439020833e-05, -0.08111...",2,109,0,True,Optimization terminated successfully.,"[0.5813468122585931, 4999.999331259375]",AISI_1020,lasso
1,4821.397135,"[[1.0, 0.0], [-0.00024284094807400458, -0.0811...",2,109,0,True,Optimization terminated successfully.,"[4.490663710540175, 4999.999331259375]",AISI_1045,lasso
1,5809.592593,"[[1.0, 0.0], [-0.00023491467972090992, -0.0811...",2,109,0,True,Optimization terminated successfully.,"[4.344089560111483, 4999.999331259375]",AISI_4340,lasso
1,20904.720216,"[[0.0, 1.0], [-0.0006564584556348865, -0.08111...",2,139,0,True,Optimization terminated successfully.,"[12.138053845103892, 4999.999900053729]",AISI_4340_52_HRC,lasso


In [32]:
def func(x):
  y=x[1]
  x=x[0]
  return x**2-2*y**3

test=[]

def callback_(x):
  global test
  test.append(x)
minimize(func,x0=[10,500],method="Powell",bounds=bounds,callback=callback_)

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: -249999999999.93396
       x: [ 3.710e-03  5.000e+03]
     nit: 3
   direc: [[ 1.000e+00  0.000e+00]
           [ 1.650e-04  7.598e-06]]
    nfev: 240

In [33]:
test

[array([1.01115363e-04, 4.99999985e+03]),
 array([3.27367023e-03, 4.99999999e+03]),
 array([3.709679e-03, 5.000000e+03])]

In [18]:
results_powell.to_csv("results_otim_powell.csv",sep='\t')

In [19]:
results

,rake_angle,vc,fcut_lasso,fcut_lgbm,material,vc_m/min
0,-15.0,2500.0,1422.019841,1040.572327,AISI_1020,150.0
1,-15.0,2500.0,5816.711489,5791.133299,AISI_1045,150.0
2,-15.0,2500.0,6792.855149,6265.977035,AISI_4340,150.0
3,-15.0,2500.0,22655.168641,6272.179365,AISI_4340_52_HRC,150.0
4,-15.0,2500.0,2657.087847,2135.276159,STEEL_304,150.0
...,...,...,...,...,...,...
595,15.0,6500.0,4879.579273,4856.126768,AISI_1045,390.0
596,15.0,6500.0,5874.343284,5774.886733,AISI_4340,390.0
597,15.0,6500.0,20746.368201,5884.420005,AISI_4340_52_HRC,390.0
598,15.0,6500.0,2145.821492,2148.521680,STEEL_304,390.0


In [45]:
x_iteracoes.head()

,model,material,x
0,lgbm,AISI_1020,"[[-6.308369924161681, 3763.932063670938]]"
1,lgbm,AISI_1045,"[[2.4822200151171674, 4527.864123305692], [2.7..."
2,lgbm,AISI_4340,"[[0.5714067824608188, 3763.932063670938], [1.1..."
3,lgbm,AISI_4340_52_HRC,"[[2.4421289220681515, 4527.864123305692], [0.9..."
4,lgbm,STEEL_304,"[[-3.9645374216096916, 3763.932063670938], [-3..."


In [70]:
x_iteracoes.iloc[[1]].x.values[0]

[array([2.48222002e+00, 4.52786412e+03]),
 array([2.70865829e+00, 4.52786801e+03])]

In [96]:
5000*60/1000

300.0

In [97]:
def get_coordinates(coords,init_coord=[0,3500]):
  n_iterarion=len(coords)
  oriented_coords=[]
  for i in range(-1,n_iterarion-1):
    if i==-1:
      x_start=init_coord[0]
      y_start=init_coord[1]*60/1000
    else:
      x_start=(coords[i][0])
      y_start=(coords[i][1])*60/1000
    x_end=(coords[i+1][0])
    y_end=(coords[i+1][1])*60/1000
    oriented_coords.append([x_end,y_end,x_start,y_start])

  return oriented_coords


In [99]:
get_coordinates(
    x_iteracoes[(x_iteracoes["material"]==material)&(x_iteracoes["model"]=="lasso")].x.values[0])

[[3.9203484685557264, 299.99996441565474, 0, 210.0],
 [3.920560181058449,
  299.9999575503757,
  3.9203484685557264,
  299.99996441565474]]

In [105]:
oriented_coords

[[0.5714067824608188, 225.83592382025628, 0, 210.0],
 [1.1671165780912343,
  245.5981095001165,
  0.5714067824608188,
  225.83592382025628],
 [1.375661608804293,
  248.60089858782666,
  1.1671165780912343,
  245.5981095001165]]

In [113]:
results

,rake_angle,vc,fcut_lasso,fcut_lgbm,material,vc_m/min
0,-15.0,150.0,1422.019841,1040.572327,AISI_1020,150.0
1,-15.0,150.0,5816.711489,5791.133299,AISI_1045,150.0
2,-15.0,150.0,6792.855149,6265.977035,AISI_4340,150.0
3,-15.0,150.0,22655.168641,6272.179365,AISI_4340_52_HRC,150.0
4,-15.0,150.0,2657.087847,2135.276159,STEEL_304,150.0
...,...,...,...,...,...,...
595,15.0,390.0,4879.579273,4856.126768,AISI_1045,390.0
596,15.0,390.0,5874.343284,5774.886733,AISI_4340,390.0
597,15.0,390.0,20746.368201,5884.420005,AISI_4340_52_HRC,390.0
598,15.0,390.0,2145.821492,2148.521680,STEEL_304,390.0


In [ ]:
import plotly.graph_objects as go

for model in ["lasso","lgbm"]:
  for count,material in enumerate(results["material"]):
    countor=pd.pivot_table(results[(results["material"]==material)],
                          columns="rake_angle",
                          index="vc",
                          values=f"fcut_{model}")
    vc=countor.index
    rake_angle=countor.columns
    fcut=countor.values

    fig = go.Figure(data =go.Contour(z=fcut,x=rake_angle, y=vc, reversescale=True,
                                     zmin=fcut.min().min(),
                                     zmax=fcut.max().max()))

    fig.update_layout(
                    title=f"Curvas de nível para o modelo {model} para material {material}",
                    xaxis_title="Ângulo de saída [°]",
                    legend_title="Força de corte [N]",
                    yaxis_title="Velocidade de corte [m/min]",
      )

    df=x_iteracoes[(x_iteracoes["material"]==material)&(x_iteracoes["model"]==model)].x.values[0]
    oriented_coords=get_coordinates(coords=df)
    list_of_all_arrows = []
    for x0,y0,x1,y1 in oriented_coords:
        arrow = go.layout.Annotation(dict(
                        x=x0,
                        y=y0,
                        xref="x", yref="y",
                        text="",
                        showarrow=True,
                        axref="x", ayref='y',
                        ax=x1,
                        ay=y1,
                        arrowhead=3,
                        arrowwidth=1.5,
                        arrowcolor='rgb(255,51,0)',)
                    )
        list_of_all_arrows.append(arrow)
    fig.update_layout(annotations=list_of_all_arrows)

    fig.show()

    #if count>1:
      #break